# Set up
### Goal

1. Build a predictive model with XGBoost
2. Deploy that model to Triton Inference Server
3. Make predictions

### Architecture

* Host: GPU (P40 or V100) and CUDA running on Ubuntu 20.4 with plenty of disk space
* Triton Container from NGC (open ports 8000, 8001, 8002)
* PyTorch Container from NGC (open port 8888)

## Set up Host

See host document

## Set up Docker network

In [ ]:
sudo docker network create tritonnet

## Pull docker containers

In [ ]:
# Triton
sudo docker pull nvcr.io/nvidia/tritonserver:22.03-py3
sudo docker run --gpus=all -d -p 8000:8000 -p 8001:8001 -p 8002:8002 --network=tritonnet -v /home/azureuser/model_repository:/models \
    --name tritonserver nvcr.io/nvidia/tritonserver:22.03-py3 tritonserver --model-repository=/models

# PyTorch
sudo docker volume create volume1
sudo docker pull nvcr.io/nvidia/pytorch:22.03-py3
sudo docker run --gpus=all -t -d -p 8888:8888 --ipc=host --ulimit memlock=-1 --ulimit stack=67108864 --network=tritonnet \
    --mount source=volume1,destination=/workspace/triton --name pytorch nvcr.io/nvidia/pytorch:22.03-py3

## Start Jupyter Lab

In [ ]:
sudo docker exec -it pytorch /bin/bash
nohup jupyter-lab --NotebookApp.token='' --no-browser --port=8888 &
exit

## Identify Triton IP

In [ ]:
sudo docker network inspect tritonnet